# Westeros Tutorial: Running MESSAGE in Recursive Dynamic Mode

While considering the full time horizon in a single run is useful for a long-term energy systems planning, the recursive dynamic approach can provide insights that reflect the relevant foresight windows decision makers can have in making adaptive and robust planning.

In addition to running the model with the perfect foresights, MESSAGE can also be run using the recursive dynamic approach. Here, the model is run iteratively throughout the periods with key decision variables from the previous periods, or iterations, are fixed so that new informations do not alter decisions that are already made in the previous periods. These variables
include $CAP$ , $CAP\_NEW$, $ACT$, and $EXT$. 

In this tutorial, we will implement the recursive dynamic mode on Westeros baseline scenario. Hence, before we start, we have to make sure that we can successfully run the baseline scenarion (``westeros_baseline.ipynb``).

Let's start with importing all the libraries we need and connect to the `ixmp` platform.

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df
from message_ix.tools.add_tech import add_tech


%matplotlib inline

mp = ixmp.Platform("local")

In [2]:
mp.scenario_list()

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
0,GENIE_sandbox,1000f_adaptive-learning,MESSAGE,1,0,pratama,2024-10-06 15:27:10.426000,pratama,2024-10-06 15:30:32.103000,None,None,clone Scenario from 'GENIE_sandbox|1000f_no-le...,6
1,GENIE_sandbox,1000f_adaptive_no-learning,MESSAGE,1,0,pratama,2024-10-06 14:21:41.226000,pratama,2024-10-06 14:24:15.408000,None,None,clone Scenario from 'GENIE_sandbox|1000f_no-le...,10
2,GENIE_sandbox,1000f_no-learning,MESSAGE,1,0,pratama,2024-06-25 18:09:08.760000,pratama,2024-06-25 18:14:26.261000,None,None,Import from GENIE_dac-sandbox_1000f.xlsx,1
3,Westeros Electrified,baseline,MESSAGE,1,0,pratama,2024-08-07 14:00:23.973000,pratama,2024-08-07 14:00:25.410000,None,None,basic model of Westeros electrification,1
4,Westeros Electrified,baseline_recursive-dynamic,MESSAGE,1,0,pratama,2024-10-04 12:48:51.676000,pratama,2024-10-04 12:48:53.469000,None,None,clone Scenario from 'Westeros Electrified|base...,48
5,Westeros Electrified,emission-bound_adaptive,MESSAGE,1,0,pratama,2024-10-04 12:48:53.656000,pratama,2024-10-04 12:48:54.627000,None,None,clone Scenario from 'Westeros Electrified|emis...,45
6,Westeros Electrified,emission-bound_adaptive-learning,MESSAGE,1,0,pratama,2024-10-04 12:48:54.672000,pratama,2024-10-04 12:48:56.059000,None,None,clone Scenario from 'Westeros Electrified|emis...,46
7,Westeros Electrified,emission_bound,MESSAGE,1,0,pratama,2024-08-07 14:03:28.466000,pratama,2024-08-07 14:03:29.889000,None,None,clone Scenario from 'Westeros Electrified|base...,1


## `baseline` scenario with recursive dynamic mode

In [3]:
# cloning baseline scenario and crate 
model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="1000f_no-learning")
scen = base.clone(
    model,
    "1000f_adaptive_no-learning",
    "1000f scenario without learning",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

**Time to Solve the Model**

In perfect foresight mode, the solve statement we add is `scen.solve()` without any additional arguments. By default, this will tell MESSAGE to run in the perfect foreseight mode. To run MESSAGE using the recursive dynamic approach, we need to add `gams_args =["--foresight=X"]` argument to the solve statement, with `X` being the length of the foresight windows. This will pass the argument directly to `GAMS`, overiding the default values set in `MESSAGE_master.gms` and `model_setup.gms` scripts. Here, let's use `X=1` as an example. 

Similarly, technology cost learning module can be activated by setting `learningmode` value to 1. This can also be done by passing `"--learningmode=1"` GAMS argument in the solve statement, i.e., `gams_args =["--foresight=X","--learningmode=1"]`. It is important to note that the learning module lives under the recursive dynamic module. Hence, this module needs to be active if learning module is used.

In [4]:
scen.commit(comment="Sandbox 1000f scenario with adaptive optimization run")
scen.set_as_default()

scen.solve(gams_args = ["--foresight=0"],
           solve_options={'scaind':'0'}
          )

scen.var("OBJ")["lvl"]

2242408.5

## `emission_bound` scenario with adaptive optimisation and learning mode

In [5]:
# cloning baseline scenario and crate 
learning_scen = base.clone(
    model,
    "1000f_adaptive-learning",
    "introducing adaptive optimisation with learning",
    keep_solution=False,
)
learning_scen.check_out()

year_df = learning_scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = learning_scen.set("year")
country = "Westeros"

In [6]:
learning_pars = {
    "coal_ppl": {
        "alpha"          : 0.22,
        "beta_unit"      : 0.21,
        "beta_proj"      : 0.0,
        "gamma_unit"     : 0.43,
        "gamma_proj"     : 0.00,
        "inv_cost_refidx": 1,
        "knref_unit"     : 4700,
        "sizeref_unit"   : 0.64,
        "sizeref_proj"   : 1.49,
    },
    "wind_ppl": {
        "alpha"          : 0.07,
        "beta_unit"      : 0.21,
        "beta_proj"      : 0.0,
        "gamma_unit"     : 0.002,
        "gamma_proj"     : 0.00,
        "inv_cost_refidx": 1,
        "knref_unit"     : 448559,
        "sizeref_unit"   : 0.003,
        "sizeref_proj"   : 26,
    }
    
}

for key,val in learning_pars.items():
    
    learning_scen.add_set("newtec",key)
    
    for key1,val1 in val.items():
        df_par = make_df(key1,
                         technology = key,
                         value = val1,
                         unit = "-"
                        )
        learning_scen.add_par(key1,df_par)
        df_par

In [7]:
learning_scen.par("alpha")

,technology,value,unit
0,coal_ppl,0.22,-
1,wind_ppl,0.07,-


In [8]:
learning_scen.commit(comment="Introducing recursive dynamic mode in MESSAGE")
learning_scen.set_as_default()

learning_scen.solve(gams_args = ["--foresight=13","--learningmode=1"],
                    solve_options={'scaind':'0'})

NameError: name 'bound_scen' is not defined

In [9]:
print("OBJ without learning:", scen.var("OBJ")["lvl"])
print("OBJ with learning   :", learning_scen.var("OBJ")["lvl"])

OBJ without learning: 2242408.5
OBJ with learning   : 2264832.75


### Let's compare the results

In [ ]:
scenarios = [bound_scen, learning_scen]
labels = ["without learning", "with learning"]

for s in range(len(scenarios)):
    df = (scenarios[s].var("CAP", {"node_loc":"Westeros","technology": "wind_ppl", "year_act":act_years})
          .drop(["node_loc","technology","year_vtg"], axis=1).set_index("year_act").groupby(level=0).sum()[["lvl"]])
    tick_loc = -1.5 if s == 0 else 1.5
    plt.bar(df.index+tick_loc,df["lvl"], width=3, label=labels[s])
plt.xticks(df.index)
plt.ylabel("wind_ppl capacity")
plt.legend()
plt.show()

## With learning

## Without learning

## Parameters and variables check

## Close the connection to the database

In [10]:
mp.close_db()